In [1]:
import os

project = {
    "E-Waste-Segregation/src/data_loader.py": """from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
import numpy as np

def load_dataset(data_dir, classes=None, images_per_class=None, image_size=(128,128), as_gray=True):
    data_dir = Path(data_dir)
    if classes is None:
        classes = sorted([p.name for p in data_dir.iterdir() if p.is_dir()])
    X_list, y_list, class_names = [], [], []
    for idx, cls in enumerate(classes):
        cls_dir = data_dir / cls
        imgs = sorted([p for p in cls_dir.iterdir() if p.is_file()])
        if images_per_class:
            imgs = imgs[:images_per_class]
        for img_path in imgs:
            try:
                img = imread(str(img_path), as_gray=as_gray)
                img = resize(img, image_size, anti_aliasing=True)
                X_list.append(img)
                y_list.append(idx)
            except:
                continue
        class_names.append(cls)
    return np.stack(X_list, axis=0), np.array(y_list), class_names
""",

    "E-Waste-Segregation/src/preprocessing.py": """import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

def flatten_images(X):
    return X.reshape(X.shape[0], -1)

def scale_features(X_train, X_val=None, scaler_path=None):
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s = scaler.transform(X_val) if X_val is not None else None
    if scaler_path: joblib.dump(scaler, scaler_path)
    return scaler, X_train_s, X_val_s
""",

    "E-Waste-Segregation/src/model_train.py": """import joblib
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def train_random_forest(X, y, model_path, n_estimators=200, test_size=0.2):
    X = np.asarray(X); y = np.asarray(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y)
    clf = RandomForestClassifier(n_estimators=n_estimators, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    joblib.dump(clf, model_path)
    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "report": classification_report(y_test, y_pred, output_dict=True),
        "confusion": confusion_matrix(y_test, y_pred)
    }
""",

    "E-Waste-Segregation/src/model_evaluate.py": """import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, class_names):
    sns.heatmap(cm, annot=True, cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted"); plt.ylabel("True"); plt.show()

def print_classification_report(report_dict):
    import json; print(json.dumps(report_dict, indent=2))
""",

    "E-Waste-Segregation/src/predict.py": """import joblib
from skimage.io import imread
from skimage.transform import resize

def load_model(p): return joblib.load(p)

def preprocess_single_image(img_path, image_size=(128,128), as_gray=True):
    img = imread(img_path, as_gray=as_gray)
    img = resize(img, image_size, anti_aliasing=True)
    return img.reshape(1, -1)

def predict_single(model, img_path, scaler=None, class_names=None):
    x = preprocess_single_image(img_path)
    if scaler: x = scaler.transform(x)
    pred = model.predict(x)[0]
    return class_names[pred] if class_names else int(pred)
""",

    "E-Waste-Segregation/src/utils.py": """import os
def ensure_dir(path): os.makedirs(path, exist_ok=True)
def list_class_folders(data_dir): return sorted(os.listdir(data_dir))
""",

    "E-Waste-Segregation/src/colab_helper.py": """from google.colab import drive
from pathlib import Path

def mount_drive(): drive.mount('/content/drive')
def get_data_dir(path):
    p = Path(path)
    if not p.exists(): raise FileNotFoundError(f"{p} not found")
    return str(p)
""",

    "E-Waste-Segregation/notebooks/colab_notebook_cells.md": """# Colab notebook cells

### 1) Mount drive
from src.colab_helper import mount_drive, get_data_dir
mount_drive()
DATA_DIR = get_data_dir('/content/drive/MyDrive/E_waste_dataset')

### 2) Load images
from src.data_loader import load_dataset
X, y, class_names = load_dataset(DATA_DIR, image_size=(128,128), as_gray=True)

### 3) Preprocess
from src.preprocessing import flatten_images, scale_features
X_flat = flatten_images(X)
scaler, X_scaled, _ = scale_features(X_flat)

### 4) Train model
from src.model_train import train_random_forest
res = train_random_forest(X_scaled, y, '/content/drive/MyDrive/e_waste_model.joblib')

### 5) Evaluate
from src.model_evaluate import plot_confusion_matrix, print_classification_report
print_classification_report(res["report"])
plot_confusion_matrix(res["confusion"], class_names)
""",

    "E-Waste-Segregation/requirements.txt": """scikit-learn
scikit-image
joblib
matplotlib
seaborn
numpy
""",

    "E-Waste-Segregation/README.md": """# E-Waste Segregation using Random Forest

A complete machine-learning pipeline to classify images of electronic waste into categories:
- mobile
- battery
- charger
- PCB
- laptop

Includes:
- data loading
- preprocessing
- model training
- evaluation
- prediction scripts
""",

    "E-Waste-Segregation/statement.md": """# Project Statement

This project classifies e-waste images using a Random Forest trained on flattened image pixels.
""",
}

# --- Write files ---
for path, content in project.items():
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w") as f:
        f.write(content)

print("Project created successfully!")


Project created successfully!
